This is for the 'part 4 - becoming a backprop ninja' lecture. He taked issue w/ us (i think he's feeling 'blindly') calling 'loss.backward' and so using PyTorch's autograd functionality to get our weights. He thinks it's important and useful for us to understand what's going on, as he writes about in https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b. 